In [ ]:
##Retail.py program

In [ ]:
from pyspark.context import SparkContext
from pyspark.sql import HiveContext, SQLContext,SparkSession
from pyspark.sql import functions as F
from datetime import date,timedelta
from datetime import datetime
sc = SparkContext(appName = "Combined_Table")
sql_context = HiveContext(sc)
spark = SparkSession(sc)
spark = (SparkSession.builder.enableHiveSupport().getOrCreate())


In [ ]:
spark 
import pandas as pd
import os
import numpy as np
import datetime
import time
import glob
import os
from numpy import nan
from pyspark.sql import functions as F

pd.set_option('display.width', 150)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
exec(open('/home/thural2/THURAL2/thural2/record.py').read())


In [ ]:
####################### FIRST PROCESS THE GROSS VOLUME SEGMENTATION METRICS###########################################
cred_ret=spark.sql("select * from anp_cabbtdct1_final.psa_elig_retail")
cred_ret_filter=cred_ret.toPandas()
cred_ret_filter.head(10)

In [ ]:
am_ret=cred_ret_filter.groupby(by=['psa_month','metric_name','am_cost_center',
                                      'am_cost_center_name','am_cost_center_full_name',
                                      'sm_cost_center','sm_cost_center_name',
                                      'record_date'],as_index=False).aggregate({
                    'elig_ind_final': 'sum'
                     })
am_ret.rename(columns = {"elig_ind_final":'count_segment'}, inplace = True)

##transpose data
am_ret1=am_ret.pivot_table(index=['metric_name','am_cost_center','am_cost_center_name',
                                        'am_cost_center_full_name',
                                        'sm_cost_center','sm_cost_center_name',
                                        'record_date'], columns=['psa_month'],
                     values='count_segment', aggfunc='first').reset_index().fillna(0)

In [ ]:
########## CREATE ALL OF THE MONTHS
##rename columns **************CHANGE FOR FUTURE
cols = ['november','december','january','february','march','april','may','june','july',
        'august','september','october']
net_a = pd.DataFrame(list, index=['a',], columns=cols)

am_ret1.rename(columns = {'11':'november', '12': 'december',
                       '1':'january','2': 'february', '3':'march', '4': 'april',
                       '5':'may', '6': 'june','7':'july', '8': 'august',
                       '9':'september', '10': 'october'}, inplace = True) 

complementary = [c for c in net_a if c not in am_ret1]

am_ret1.reset_index(drop=True, inplace=True)
am_ret2= pd.concat([net_a[complementary], am_ret1], axis=1, join='outer')

##delete the extra filler row
am_ret2= am_ret2.drop(labels='a', axis=0)
##convert to float
am_ret2 = am_ret2.astype({'november': float,'december': float,
                                'january': float, 'february': float, 'march': float,
                                'april': float, 'may': float,'june':float,'july': float,
                                'august': float, 'september':float,'october': float})

am_ret2['scorecard_filter']='Retail'
am_ret2['level']='AM'
am_ret2['acf2_id']=''
am_ret2['employee_name']=am_ret2['am_cost_center_full_name']

## order relevant vars for amsb_cred3
am_ret3=am_ret2[['metric_name','level','acf2_id','employee_name',
                'november','december','january','february','march','april','may','june','july',
                'august','september','october','am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
## also group by to get the total widgets
am_ret4=am_ret3.groupby(by=['acf2_id','employee_name','am_cost_center',
                                  'am_cost_center_name','am_cost_center_full_name',
                                  'sm_cost_center','sm_cost_center_name',
                                  'scorecard_filter','level','record_date'],as_index=False).aggregate({
                        'november': 'sum',
                        'december': 'sum',
                        'january': 'sum',
                        'february': 'sum',
                        'march': 'sum',
                        'april': 'sum',
                        'may': 'sum',
                        'june': 'sum',
                        'july': 'sum',
                        'august': 'sum',
                        'september': 'sum',
                        'october': 'sum'
                     })

am_ret4['metric_name']='Total SBB Credit Widgets'

## order relevant vars for amsb_cred4
am_ret5=am_ret4[['metric_name','level','acf2_id','employee_name',
                'november','december','january','february','march','april','may','june','july',
                'august','september','october',
               'am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
### APPEND TOTAL TO THE OTHER SUB-CATEGORIES
am_ret6=pd.concat([am_ret5,am_ret3],axis=0)

In [ ]:
## NOW create QTD and YTD variables
newvarfunc(am_ret6)

## replace 0 with NaN so median only counts non-zeros
am_ret6=am_ret6.replace(0.0, np.nan)

## CREATE THE QTD BENCHMARK ("quarter" is a set variable (current))
## Q1
ret_am_median=am_ret6.groupby(['metric_name'], as_index=False)['q1'].median()
##rename median
ret_am_median.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
### merge to dataframe
ret_am_all3=pd.merge(am_ret6,ret_am_median,on=['metric_name'],
                   how="left")

## Q2
ret_am_median1=am_ret6.groupby(['metric_name'], as_index=False)['q2'].median()
##rename median
ret_am_median1.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
### merge to dataframe
ret_am_all4=pd.merge(ret_am_all3,ret_am_median1,on=['metric_name'],
                   how="left")

## Q3
ret_am_median2=am_ret6.groupby(['metric_name'], as_index=False)['q3'].median()
##rename median
ret_am_median2.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
### merge to dataframe
ret_am_all5=pd.merge(ret_am_all4,ret_am_median2,on=['metric_name'],
                   how="left")

## Q4
ret_am_median3=am_ret6.groupby(['metric_name'], as_index=False)['q4'].median()
##rename median
ret_am_median3.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
### merge to dataframe
ret_am_all6=pd.merge(ret_am_all5,ret_am_median3,on=['metric_name'],
                   how="left")


## CREATE THE YTD BENCHMARK ("quarter" is a set variable (current))
ret_am_ytdmedian=am_ret6.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
ret_am_ytdmedian.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)
### merge to dataframe
ret_am_all7=pd.merge(ret_am_all6,ret_am_ytdmedian,on=['metric_name'],
                   how="left")

ret_am_all8=ret_am_all7[['metric_name','level','acf2_id','employee_name',
                'november','december','january','february','march','april','may','june','july',
                'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
               'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
               'am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]
ret_am_all8.info()


In [ ]:
##############################  GROUP BY SM (Retail FILTER)##############################
ret_sm=smgrpfunc(ret_am_all8)

##include the LEVEL variable
ret_sm['level']='SM'
ret_sm['acf2_id']=''
ret_sm['scorecard_filter']='Retail'
ret_sm['employee_name']=ret_sm['sm_cost_center_name']
ret_sm['am_cost_center']=''
ret_sm['am_cost_center_name']=''
ret_sm['am_cost_center_full_name']=''

##CREATE THE AM QUARTER AND YTD VARIABLES
newvarfunc(ret_sm)

## replace 0 with NaN so median only counts non-zeros
ret_sm=ret_sm.replace(0.0, np.nan)

## CREATE THE QTD BENCHMARK ("quarter" is a set variable (current))
## Q1
ret_sm_median=ret_sm.groupby(['metric_name'], as_index=False)['q1'].median()
##rename median
ret_sm_median.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
### merge to dataframe
ret_sm_all3=pd.merge(ret_sm,ret_sm_median,on=['metric_name'],
                   how="left")

## Q2
ret_sm_median1=ret_sm.groupby(['metric_name'], as_index=False)['q2'].median()
##rename median
ret_sm_median1.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
### merge to dataframe
ret_sm_all4=pd.merge(ret_sm_all3,ret_sm_median1,on=['metric_name'],
                   how="left")

## Q3
ret_sm_median2=ret_sm.groupby(['metric_name'], as_index=False)['q3'].median()
##rename median
ret_sm_median2.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
### merge to dataframe
ret_sm_all5=pd.merge(ret_sm_all4,ret_sm_median2,on=['metric_name'],
                   how="left")

## Q4
ret_sm_median3=ret_sm.groupby(['metric_name'], as_index=False)['q4'].median()
##rename median
ret_sm_median3.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
### merge to dataframe
ret_sm_all6=pd.merge(ret_sm_all5,ret_sm_median3,on=['metric_name'],
                   how="left")

## CREATE THE YTD BENCHMARK ("quarter" is a set variable (current))
ret_sm_ytdmedian=ret_sm.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
ret_sm_ytdmedian.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)
### merge to dataframe
ret_sm_all7=pd.merge(ret_sm_all6,ret_sm_ytdmedian,on=['metric_name'],
                   how="left")

## keep relevant vars
ret_sm_all8=ret_sm_all7[['metric_name','level','acf2_id','employee_name',
                'november','december','january','february','march','april','may','june','july',
                'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
               'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
               'am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
######################################################################################
############################# NOW PROCESS PSA METRICS ################################
## spark sql
df_ret_fil=spark.sql("select * from anp_cabbtdct1_final.SCORECARD_RETAIL_FILTER where prod_nm in {}".format(tuple(all_prod)))
mif_ret_filter=df_ret_fil.toPandas()
mif_ret_filter.head(10)

In [ ]:
## rename the numeric months to full month name
mif_ret_filter.rename(columns = {'month11':'november', 'month12': 'december',
                       'month1':'january','month2': 'february', 'month3':'march', 'month4': 'april',
                       'month5':'may', 'month6': 'june','month7':'july', 'month8': 'august',
                       'month9':'september', 'month10': 'october'}, inplace = True)

##convert to float
mif_ret_filter = mif_ret_filter.astype({'november': float,'december': float,
                                'january': float, 'february': float, 'march': float,
                                'april': float, 'may': float,'june':float,'july': float,
                                'august': float, 'september':float,'october': float})

## filter out VOLUME
df_mif_volume=mif_ret_filter[(mif_ret_filter.bal_typ=='BM') & (mif_ret_filter.sub_typ=='A')]

## filter out WIDGETS
df_mif_widgets=mif_ret_filter[(mif_ret_filter.bal_typ=='BM') & (mif_ret_filter.sub_typ=='B')]

In [ ]:
############################# GROSS VOLUME ###########################################
## RETAIL TAB: TOTAL SBB GROSS VOLUME
sbb_gross_filter=df_mif_volume[df_mif_volume.prod_nm.isin(gross_all_tuple)]
sbb_gross_filter=sbb_gross_filter.assign(metric_name='Total SBB Gross Volume')
sbb_gross_filter=filterfunc(sbb_gross_filter)

## SBB GROSS LOC VOLUME
gross_loc_filter=df_mif_volume[df_mif_volume.prod_nm.isin(gross_loc_tuple)]
gross_loc_filter=gross_loc_filter.assign(metric_name='SBB Gross LOC Volume')
gross_loc_filter=filterfunc(gross_loc_filter)

## SBB GROSS LON VOLUME
gross_lon_filter=df_mif_volume[df_mif_volume.prod_nm.isin(gross_lon_tuple)]
gross_lon_filter=gross_lon_filter.assign(metric_name='SBB Gross LON Volume')
gross_lon_filter=filterfunc(gross_lon_filter)

## SBB GROSS PROFESSIONAL STUDENT LOC VOLUME
gross_prof_filter=df_mif_volume[df_mif_volume.prod_nm.isin(gross_student_loc_tuple)]
gross_prof_filter=gross_prof_filter.assign(metric_name='Gross Professional Student LOC')
gross_prof_filter=filterfunc(gross_prof_filter)

### APPEND ALL THE DATAFRAME FILTERS
gross_all=pd.concat([sbb_gross_filter,gross_loc_filter,gross_lon_filter,gross_prof_filter],axis=0)

########################## CREDIT CARDS ##########################################

## RETAIL TAB: TOTAL CREDIT CARDS
sbb_cc_filter=df_mif_widgets[df_mif_widgets.prod_nm.isin(total_cc_tuple)]
sbb_cc_filter=sbb_cc_filter.assign(metric_name='Total Credit Cards')
sbb_cc_filter=filterfunc(sbb_cc_filter)

## RETAIL TAB: BUSINESS VISA
sbb_bv_filter=df_mif_widgets[df_mif_widgets.prod_nm.isin(business_visa_cc_tuple)]
sbb_bv_filter=sbb_bv_filter.assign(metric_name='Credit Cards - Business VISA')
sbb_bv_filter=filterfunc(sbb_bv_filter)

## RETAIL TAB: BUSINESS VISA
sbb_btv_filter=df_mif_widgets[df_mif_widgets.prod_nm.isin(business_travel_tuple)]
sbb_btv_filter=sbb_btv_filter.assign(metric_name='Credit Cards - Business Travel VISA')
sbb_btv_filter=filterfunc(sbb_btv_filter)

## RETAIL TAB: AEROPLAN BUSINESS
sbb_aero_filter=df_mif_widgets[df_mif_widgets.prod_nm.isin(business_aero_tuple)]
sbb_aero_filter=sbb_aero_filter.assign(metric_name='Credit Cards - Aeroplan Business')
sbb_aero_filter=filterfunc(sbb_aero_filter)

## RETAIL TAB: BUSINESS SELECT NO FEE
sbb_nofee_filter=df_mif_widgets[df_mif_widgets.prod_nm.isin(bus_nofee_tuple)]
sbb_nofee_filter=sbb_nofee_filter.assign(metric_name='Credit Cards - Business Select No Fee')
sbb_nofee_filter=filterfunc(sbb_nofee_filter)

## RETAIL TAB: BUSINESS SELECT FEE
sbb_fee_filter=df_mif_widgets[df_mif_widgets.prod_nm.isin(bus_fee_tuple)]
sbb_fee_filter=sbb_fee_filter.assign(metric_name='Credit Cards - Business Select Fee')
sbb_fee_filter=filterfunc(sbb_fee_filter)

### APPEND ALL THE DATAFRAME FILTERS
cc_all=pd.concat([sbb_cc_filter,sbb_bv_filter,sbb_btv_filter,sbb_aero_filter,
                  sbb_nofee_filter,sbb_fee_filter],axis=0)

########################## MERCHANT SOLUTIONS ##########################################

## RETAIL TAB: TOTAL MERCHANT SOLUTIONS
merchant_all=df_mif_widgets[df_mif_widgets.prod_nm.isin(total_mer_tuple)]
merchant_all=merchant_all.assign(metric_name='Total Merchant Solutions')
merchant_all=filterfunc(merchant_all)

################################ CMS ###############################################

## RETAIL TAB: TOTAL CMS
cms_all=df_mif_widgets[df_mif_widgets.prod_nm.isin(total_cms_tuple)]
cms_all=cms_all.assign(metric_name='Total Cash Management Services (including RDC)')
cms_all=filterfunc(cms_all)

################################ COMMERCIAL VOLUME ###################################

## RETAIL TAB: TOTAL COMMERCIAL VOLUME
sbb_com_filter=df_mif_volume[df_mif_volume.prod_nm.isin(total_com_tuple)]
sbb_com_filter=sbb_com_filter.assign(metric_name='ALL Total Commercial Volume')
sbb_com_filter=filterfunc(sbb_com_filter)

## RETAIL TAB: COMMERCIAL DEPOSIT VOLUME
sbb_dep_filter=df_mif_volume[df_mif_volume.prod_nm.isin(com_deposit_tuple)]
sbb_dep_filter=sbb_dep_filter.assign(metric_name='All Commercial Deposit Volume')
sbb_dep_filter=filterfunc(sbb_dep_filter)

## RETAIL TAB: COMMERCIAL LOAN VOLUME
sbb_loan_filter=df_mif_volume[df_mif_volume.prod_nm.isin(com_loan_tuple)]
sbb_loan_filter=sbb_loan_filter.assign(metric_name='All Commercial Loan Volume')
sbb_loan_filter=filterfunc(sbb_loan_filter)

### APPEND ALL THE DATAFRAME FILTERS
com_all=pd.concat([sbb_com_filter,sbb_dep_filter,sbb_loan_filter],axis=0)

################################ NEW ACCOUNT OPENINGS ###############################

## RETAIL TAB: TOTAL ACCOUNT OPENINGS
sbb_open_filter=df_mif_widgets[df_mif_widgets.prod_nm.isin(total_new_tuple)]
sbb_open_filter=sbb_open_filter.assign(metric_name='Total New SBB Chequing Accounts')
sbb_open_filter=filterfunc(sbb_open_filter)

################################ TD SECURITIES DIRECT TRADE ###########################

## RETAIL TAB: TOTAL TD SECURITIES DIRECT TRADE
sbb_trade_filter=df_mif_widgets[df_mif_widgets.prod_nm.isin(total_trade_tuple)]
sbb_trade_filter=sbb_trade_filter.assign(metric_name='TD Securities Direct Trade')
sbb_trade_filter=filterfunc(sbb_trade_filter)

################################ BCP WIDGETS ##########################################

## RETAIL TAB: TOTAL BCP WIDGETS
sbb_bcp_filter=df_mif_widgets[df_mif_widgets.prod_nm.isin(total_bcp_tuple)]
sbb_bcp_filter=sbb_bcp_filter.assign(metric_name='Business Credit Protection(BCP) Widgets')
sbb_bcp_filter=filterfunc(sbb_bcp_filter)

############################# MUR MORTGAGE VOLUME######################################

## RETAIL TAB: MUR MORTGAGE VOLUME
sbb_mur_filter=df_mif_volume[df_mif_volume.prod_nm.isin(total_mur_tuple)]
sbb_mur_filter=sbb_mur_filter.assign(metric_name='Total MUR Mortgage Volume')
sbb_mur_filter=filterfunc(sbb_mur_filter)

################################ WEALTH REFERRAL WIDGETS #############################

## RETAIL TAB: WEALTH REFERRAL WIDGETS
#sbb_wealth_filter=df_mif_widgets[df_mif_widgets.prod_nm.isin(total_wealth_tuple)]
#sbb_wealth_filter=sbb_wealth_filter.assign(metric_name='Total Wealth Referrals')
#sbb_wealth_filter=filterfunc(sbb_wealth_filter)

########################################### BODP ############################################

## RETAIL TAB: BODP
sbb_bodp_filter=df_mif_widgets[df_mif_widgets.prod_nm.isin(total_bodp_tuple)]
sbb_bodp_filter=sbb_bodp_filter.assign(metric_name='Total BODP')
sbb_bodp_filter=filterfunc(sbb_bodp_filter)

############################ CONCATENATE ALL DATAFRAMES ###############################

### APPEND ALL THE DATAFRAME FILTERS 
ret_psa_all=pd.concat([gross_all,cc_all,merchant_all,cms_all, com_all,
                   sbb_open_filter,sbb_trade_filter,sbb_bcp_filter,
                   sbb_mur_filter,sbb_bodp_filter],axis=0)

In [ ]:
ret_psa_all.metric_name.unique()

In [ ]:
##############################  GROUP BY AM (RETAIL FILTER)##############################
ret_psa_am=amgrpfunc(ret_psa_all)

##include the LEVEL variable
ret_psa_am['level']='AM'
ret_psa_am['scorecard_filter']='Retail'
ret_psa_am['acf2_id']=''
ret_psa_am['employee_name']=ret_psa_am['am_cost_center_full_name']

##CREATE THE AM QUARTER AND YTD VARIABLES
newvarfunc(ret_psa_am)

## replace 0 with NaN so median only counts non-zeros
ret_psa_am=ret_psa_am.replace(0.0, np.nan)

## CREATE THE QTD BENCHMARK ("quarter" is a set variable (current))
## Q1
ret_psa_am_median=ret_psa_am.groupby(['metric_name'], as_index=False)['q1'].median()
##rename median
ret_psa_am_median.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
### merge to dataframe
ret_psa_am_all3=pd.merge(ret_psa_am,ret_psa_am_median,on=['metric_name'],
                   how="left")

## Q2
ret_psa_am_median1=ret_psa_am.groupby(['metric_name'], as_index=False)['q2'].median()
##rename median
ret_psa_am_median1.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
### merge to dataframe
ret_psa_am_all4=pd.merge(ret_psa_am_all3,ret_psa_am_median1,on=['metric_name'],
                   how="left")

## Q3
ret_psa_am_median2=ret_psa_am.groupby(['metric_name'], as_index=False)['q3'].median()
##rename median
ret_psa_am_median2.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
### merge to dataframe
ret_psa_am_all5=pd.merge(ret_psa_am_all4,ret_psa_am_median2,on=['metric_name'],
                   how="left")

## Q4
ret_psa_am_median3=ret_psa_am.groupby(['metric_name'], as_index=False)['q4'].median()
##rename median
ret_psa_am_median3.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
### merge to dataframe
ret_psa_am_all6=pd.merge(ret_psa_am_all5,ret_psa_am_median3,on=['metric_name'],
                   how="left")


## CREATE THE YTD BENCHMARK ("quarter" is a set variable (current))
ret_psa_am_ytdmedian=ret_psa_am.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
ret_psa_am_ytdmedian.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)
### merge to dataframe
ret_psa_am_all7=pd.merge(ret_psa_am_all6,ret_psa_am_ytdmedian,on=['metric_name'],
                   how="left")

## keep relevant vars
ret_psa_am_all8=ret_psa_am_all7[['metric_name','level','acf2_id','employee_name',
               'november','december','january','february','march','april','may','june','july',
               'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
               'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
               'am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


In [ ]:
##############################  GROUP BY SM (RETAIL FILTER)##############################
ret_psa_sm=smgrpfunc(ret_psa_am)

##include the LEVEL variable
ret_psa_sm['level']='SM'
ret_psa_sm['acf2_id']=''
ret_psa_sm['scorecard_filter']='Retail'
ret_psa_sm['employee_name']=ret_psa_sm['sm_cost_center_name']
ret_psa_sm['am_cost_center']=''
ret_psa_sm['am_cost_center_name']=''
ret_psa_sm['am_cost_center_full_name']=''

##CREATE THE AM QUARTER AND YTD VARIABLES
newvarfunc(ret_psa_sm)

## replace 0 with NaN so median only counts non-zeros
ret_psa_sm=ret_psa_sm.replace(0.0, np.nan)

## CREATE THE QTD BENCHMARK ("quarter" is a set variable (current))
## Q1
ret_psa_sm_median=ret_psa_sm.groupby(['metric_name'], as_index=False)['q1'].median()
##rename median
ret_psa_sm_median.rename(columns = {'q1':'q1_benchmark'}, inplace = True)
### merge to dataframe
ret_psa_sm_all3=pd.merge(ret_psa_sm,ret_psa_sm_median,on=['metric_name'],
                   how="left")

## Q2
ret_psa_sm_median1=ret_psa_sm.groupby(['metric_name'], as_index=False)['q2'].median()
##rename median
ret_psa_sm_median1.rename(columns = {'q2':'q2_benchmark'}, inplace = True)
### merge to dataframe
ret_psa_sm_all4=pd.merge(ret_psa_sm_all3,ret_psa_sm_median1,on=['metric_name'],
                   how="left")

## Q3
ret_psa_sm_median2=ret_psa_sm.groupby(['metric_name'], as_index=False)['q3'].median()
##rename median
ret_psa_sm_median2.rename(columns = {'q3':'q3_benchmark'}, inplace = True)
### merge to dataframe
ret_psa_sm_all5=pd.merge(ret_psa_sm_all4,ret_psa_sm_median2,on=['metric_name'],
                   how="left")

## Q4
ret_psa_sm_median3=ret_psa_sm.groupby(['metric_name'], as_index=False)['q4'].median()
##rename median
ret_psa_sm_median3.rename(columns = {'q4':'q4_benchmark'}, inplace = True)
### merge to dataframe
ret_psa_sm_all6=pd.merge(ret_psa_sm_all5,ret_psa_sm_median3,on=['metric_name'],
                   how="left")


## CREATE THE YTD BENCHMARK ("quarter" is a set variable (current))
ret_psa_sm_ytdmedian=ret_psa_sm.groupby(['metric_name'], as_index=False)['ytd'].median()
##rename median
ret_psa_sm_ytdmedian.rename(columns = {'ytd':'ytd_benchmark'}, inplace = True)
### merge to dataframe
ret_psa_sm_all7=pd.merge(ret_psa_sm_all6,ret_psa_sm_ytdmedian,on=['metric_name'],
                   how="left")

## keep relevant vars
ret_psa_sm_all8=ret_psa_sm_all7[['metric_name','level','acf2_id','employee_name',
                'november','december','january','february','march','april','may','june','july',
                'august','september','october','q1','q1_benchmark','q2','q2_benchmark','q3',
               'q3_benchmark','q4','q4_benchmark','ytd','ytd_benchmark',
               'am_cost_center','am_cost_center_name','am_cost_center_full_name',
               'sm_cost_center','sm_cost_center_name','scorecard_filter','record_date']]


### the 3 dataframes (AMSB, AM, SM) have aggregated month1-month12 and q1-q4,ytd

### APPEND ALL THE DATAFRAME FILTERS including the segmentation metrics
df_ret_all=pd.concat([ret_psa_am_all8,ret_psa_sm_all8,
                      ret_am_all8,ret_sm_all8],axis=0)


In [ ]:
## Convert pandas dataframe to Pyspark dataframe and save in table
spark.sql("set hive.exec.dynamic.partition.mode=nonstrict")
spark.createDataFrame(df_ret_all).write.mode("append").partitionBy("record_date").format("hive").saveAsTable("anp_cabbtdct1_final.SBB_SC_METRICS_RETAIL") 

In [ ]:
## END OF PROGRAM